# Running this notebook
`bazel build lib/Transforms/SO3:jupyter && ./bazel-bin/lib/Transforms/SO3/jupyter`

# SO(3), the group of rotations in 3D space
## Introduction
SO(3), the 3D rotation group, is the group of all roations about the origin of three-dimensional Euclidian space $\mathbb{R}^3$ under the operation of composition ([wiki](https://en.wikipedia.org/wiki/3D_rotation_group)). 

- A rotation about the origin preserves the origin, euclidian distance and orientation.
- Composing two rotations results in another rotation.
- Every rotation has a unique inverse rotation.
- The identity map satisfies the definition of a rotation.

Therefore the set of all rotations is a group under composition.

In [1]:
from scipy.spatial.transform import Rotation as Rot
# TODO matrices, vectors, quaternions
%load_ext autoreload
from SO3 import *
import inspect

lines = inspect.getsource(SO3.__init__)
print(lines)

RR = Rot.from_rotvec(([0, 0, 3.1415/2]))
rotation = SO3(RR.as_matrix())

    def __init__(self, R=None):
        if R is None:
            R = [[1, 0, 0], [0, 1, 0], [0, 0, 1]]
        self.R = R



## Representing rotations
TODO move this to its own notebook.

Any 3D rotation can be described as a single rotation about some axis.
While the minimum required number of parameters required too describe a rotation is 3, there are multiple ways to represent a rotation.

We can represent rotations using:

### Matrices (SO3))
Matrices are useful because:
- They allow to easily rotate vectors
- They make it easy to combine rotations

However:
- They are not concise

Matrices have 9 degrees of freedom, which means they need 6 constraints to express 3D rotations.

Rotation matrices are usually formed with column vectors. They represent a 3D orthonormal basis.

### Euler axis and angle
We can represent the rotation a a 3-vector, of which the norm is the angle.

The axis-angle representation is useful because:
- It only takes three scalars

However:
- They are undefined if the angle is 0
- They are not trivial to combine
- They suffer from gimbal lock

### Euler rotations
They split the rotation of the coordinate system into three simpler consecutive rotation. For instance in aviation:; z-y-x = yaw pitch roll.
There are thus different possible orders of rotations.

We can differentiate intrinsic (ie rotate around the axes of the object) and extrinsic (ie rotate around the axis of the coordinate frame) rotations. They are equivalent, except that the order of rotations in inverted.

### Quaternions
They form a 4 dimensional vector space.

They have mainly the following benefits:
- Compact representation and less susceptible to round-off errors
- We can combine rotations simply by using the product


In [4]:
rotation.R

array([[ 4.63267949e-05, -9.99999999e-01,  0.00000000e+00],
       [ 9.99999999e-01,  4.63267949e-05,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  1.00000000e+00]])

## SO(3) in Lie theory
The rotation group has a natural structure as a manifold for which the group operations are smoothly differentiable, so it is a Lie group.

### Lie algebra
Associated with every Lie group is its Lie algebra, a linear space of the same dimension as the Lie group, closed under a bilinear alternating product called the Lie bracket. 

The Lie algebra of SO(3) is denoted by $\mathfrak {so}(3)$ and consists of all skew-symmetric 3 × 3 matrices. 

The Lie algebra $\mathfrak {so}(3)$ is isomorphic to the Lie algebra  $\mathbb{R}^3$. Under this isomorphism, an Euleur vector $\omega \in \mathbb{R}^3$ corresponds to the linear map $\tilde{\omega} (u) = \omega \times u$.

In practice, a suitable basis to use $\mathfrak {so}(3)$ as a 3-dimensional vector space is
$$
\boldsymbol{L}_x = \begin{bmatrix}0&0&0\\0&0&-1\\0&1&0\end{bmatrix}, \quad
\boldsymbol{L}_y = \begin{bmatrix}0&0&1\\0&0&0\\-1&0&0\end{bmatrix}, \quad
\boldsymbol{L}_z = \begin{bmatrix}0&-1&0\\1&0&0\\0&0&0\end{bmatrix}.
$$
We can then identify any matrix of in this Lie algebra with an Euler vector $\omega = (x, y, z) \in \mathbb{R}^3$.

$$
\widehat{\boldsymbol{\omega}} =\boldsymbol{\omega}\cdot \boldsymbol{L} = x \boldsymbol{L}_x + y \boldsymbol{L}_y + z \boldsymbol{L}_z =\begin{bmatrix}0&-z&y\\z&0&-x\\-y&x&0\end{bmatrix} \in \mathfrak{so}(3)
$$

We can go from the skew symetric notations to euler angles and back using the skew and vee operators.

In [5]:
lines = inspect.getsource(skew)
print(lines)

lines = inspect.getsource(vee)
print(lines)

def skew(v):
    return [[0, -v[2], v[1]], [v[2], 0, -v[0]], [-v[1], v[0], 0]]

def vee(W):
    return [-W[1][2], W[0][2], -W[0][1]]



#### Logarithm map $\log : \operatorname{SO}(3) \to \mathfrak{so}(3)$
Given $R \in \operatorname{SO}(3)$, let $A = \frac{1}{2} (R - R^T)$ and let $\|A\| = \sqrt{-\frac{1}{2}\operatorname{Tr}\left(A^2\right)}$. Then the logarithm of $R$ is given by 
$$
\log R = \frac{\sin^{-1}\|A\|}{\|A\|}A
$$

In [6]:
lines = inspect.getsource(SO3.log_map)
print(lines)

    def log_map(self):
        """
        Returns the skew matrix in lie algebra associated to the
        SO3 rotation matrix R
        """
        R = self.R
        A = (R - R.T) / 2
        norm = np.sqrt(-np.trace(A @ A) / 2)
        skew_matrix = ((np.arcsin(norm)) / norm) * A
        return so3(vee(skew_matrix))



In [7]:
RR = Rot.from_rotvec(np.pi/2 * np.array([0, 0, 1]))
R = RR.as_matrix()
rotation = SO3(R)
so = rotation.log_map()
so, RR.as_euler("xyz")

(so3: [-0.0, 0.0, 1.5707963267948966],
 array([0.        , 0.        , 1.57079633]))

#### Exponential map $\exp : \mathfrak{so}(3) \to \operatorname{SO}(3)$
Since $\operatorname{SO}(3)$ is a matrix group, the exponential map is the matrix exponential series
$$
\begin{cases}
\exp : \mathfrak{so}(3) \to \operatorname{SO}(3) \\
A \mapsto e^A = \sum_{k=0}^\infty \frac{1}{k!} A^k
 = I + A + \tfrac{1}{2} A^2 + \cdots.
\end{cases}
$$
$A \in \mathfrak{so}(3)$ being by definition a skew-symetric matrix.

In [8]:
lines = inspect.getsource(so3.exp_map_generic)
print(lines)

    def exp_map_generic(self):
        """
        Simply computes the matrix exponent.
        We should iterate to infinity but we stop at 50.
        """
        A = skew(self.w)
        exp = np.eye(3)
        for k in range(1, 50):
            exp += (1 / np.math.factorial(k)) * np.linalg.matrix_power(A, k)
        return SO3(exp)



In [9]:
so.exp_map_generic(), R

(SO3: [[ 4.26480464e-17 -1.00000000e+00  0.00000000e+00]
  [ 1.00000000e+00  4.26480464e-17  0.00000000e+00]
  [ 0.00000000e+00  0.00000000e+00  1.00000000e+00]],
 array([[ 2.22044605e-16, -1.00000000e+00,  0.00000000e+00],
        [ 1.00000000e+00,  2.22044605e-16,  0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00,  1.00000000e+00]]))

However as every $A \in \mathfrak{so}(3)$ is associated to a vector $\omega = \theta u$, a direct exponentiation is possible, and yields

$$
\begin{align}
 \exp(\tilde{\boldsymbol{\omega}})
   &= \exp(\theta(\boldsymbol{u\cdot L}))
    = \exp\left(\theta \begin{bmatrix} 0 & -z & y \\ z & 0 & -x \\ -y & x & 0 \end{bmatrix}\right)\\[4pt]
   &= \boldsymbol{I} + 2cs(\boldsymbol{u\cdot L}) + 2s^2 (\boldsymbol{u \cdot L})^2 \\[4pt]
   &= \begin{bmatrix}
        2 \left(x^2 - 1\right) s^2 + 1 & 2 x y s^2 - 2 z c s & 2 x z s^2 + 2 y c s \\
        2 x y s^2 + 2 z c s & 2 \left(y^2 - 1\right) s^2 + 1 & 2 y z s^2 - 2 x c s \\
        2 x z s^2 - 2 y c s & 2 y z s^2 + 2 x c s & 2 \left(z^2 - 1\right) s^2 + 1
      \end{bmatrix},
\end{align}
$$

In [10]:
lines = inspect.getsource(so3.exp_map_euler)
print(lines)

    def exp_map_euler(self):
        """
        SO3 specific implementation
        """
        w = np.array(self.w)
        theta = np.linalg.norm(w)
        w = w / theta
        x = w[0]
        y = w[1]
        z = w[2]

        c = np.cos(theta / 2)
        s = np.sin(theta / 2)

        exp = [
            [
                2 * (x**2 - 1) * s**2 + 1,
                2 * x * y * (s**2) - 2 * z * c * s,
                2 * x * z * s**2 + 2 * y * c * s,
            ],
            [
                2 * x * y * s**2 + 2 * z * c * s,
                2 * (y**2 - 1) * s**2 + 1,
                2 * y * z * s**2 - 2 * x * c * s,
            ],
            [
                2 * x * z * s**2 - 2 * y * c * s,
                2 * y * z * s**2 + 2 * x * c * s,
                2 * (z**2 - 1) * s**2 + 1,
            ],
        ]
        return SO3(np.array(exp))



In [11]:
so.exp_map_euler(), R

(SO3: [[ 2.22044605e-16 -1.00000000e+00  0.00000000e+00]
  [ 1.00000000e+00  2.22044605e-16  0.00000000e+00]
  [-0.00000000e+00  0.00000000e+00  1.00000000e+00]],
 array([[ 2.22044605e-16, -1.00000000e+00,  0.00000000e+00],
        [ 1.00000000e+00,  2.22044605e-16,  0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00,  1.00000000e+00]]))

# Generating random rotation
## Using the von Mises Fisher distribution
We can draw random points on the 3D sphere using the von Mises Fisher distribution and consider that they lie on the algebra of SO3. 

In [10]:
from vonMisesFisher.vonMisesFisher import vMF
from lib.Transforms.SO3 import SO3
v = vMF.samplevMF(kappa = 50)
v_in_algebra = SO3.so3(v)
V_SO3 = v_in_algebra.exp_map_euler()
V_SO3

SO3: [[ 0.56400136 -0.77885805 -0.27437675]
 [ 0.82229032  0.56020144  0.10006484]
 [ 0.07576995 -0.28205405  0.95640181]]